**Sample ID**: Gemini_Apps_MT_ID_17

**Query**:

Schedule Alarm for 6 AM tomorrow

**DB Type**: Base Case

**Case Description**: Initially, there is no alarm set for 6 AM, Do Not Disturb is not enabled, and no recent WhatsApp messages are received from Riya. Finally, an alarm for 6 AM tomorrow is scheduled, Do Not Disturb remains disabled, and a phone call is placed to Riya on her mobile.

```
<multiturn info>
Query 2: Turn on Do Not Disturb (Follow Up Request)
Query 3: Cancel previous DND request and check for new WhatsApp messages from Riya (Follow Up Request)
Query 4: Call her (Follow Up Request)
Call Clarification: Call Riya's mobile (Ambiguity Resolution)
Channel Info: Via Phone (Information Gathering)
</multiturn info>
```

**Global/Context Variables:**


**Datetime Context Variables**:

- current_date = "2025-07-21"


**APIs:**
- clock
- notifications
- phone
- google_people

**Databases:**

# Set Up

## Download relevant files

In [ ]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Version to download
VERSION = "0.1.4"  # This will be replaced dynamically

# Define paths
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

# Google Drive Folder ID where versioned APIs zip files are stored
APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'

# List of items to extract from the zip file
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/', 'Schemas/']

# Clean up existing directories and files
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Authenticate and create the drive service
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Helper function to download a file from Google Drive
def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    """Downloads a file from Google Drive"""
    destination = output_path
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(destination, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")

# 1. List files in the specified APIs folder
print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None

try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    for file in files:
        file_name = file.get('name', '')
        if file_name.lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file.get('id')
            print(f"Found matching file: {file_name} (ID: {apis_file_id})")
            break

except Exception as e:
    print(f"An error occurred while listing files in Google Drive: {e}")

if not apis_file_id:
    print(f"Error: Could not find APIs zip file with version {VERSION} in the specified folder.")
    sys.exit("Required APIs zip file not found.")

# 2. Download the found APIs zip file
print(f"Downloading APIs zip file with ID: {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH, file_name=f'APIs_V{VERSION}.zip')

# 3. Extract specific items from the zip file to /content
print(f"Extracting specific items from {ZIP_PATH} to {CONTENT_DIR}...")
try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_contents = zip_ref.namelist()

        for member in zip_contents:
            extracted = False
            for item_prefix in ITEMS_TO_EXTRACT:
                if member == item_prefix or member.startswith(item_prefix):
                    zip_ref.extract(member, CONTENT_DIR)
                    extracted = True
                    break

except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {ZIP_PATH} is not a valid zip file.")
    sys.exit("Invalid zip file downloaded.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    sys.exit("Extraction failed.")

# 4. Clean up
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# 5. Add APIs to path
if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)
else:
    print(f"Error: APIS directory not found at {APIS_DIR} after extraction. Cannot add to path.")

# 6. Quick verification
# Check for the presence of the extracted items
verification_paths = [APIS_DIR, DBS_DIR, SCRIPTS_DIR]
all_present = True
print("\nVerifying extracted items:")
for path in verification_paths:
    if os.path.exists(path):
        print(f"✅ {path} is present.")
    else:
        print(f"❌ {path} is MISSING!")
        all_present = False

if all_present:
    print(f"\n✅ Setup complete! Required items extracted to {CONTENT_DIR}.")
else:
    print("\n❌ Setup failed! Not all required items were extracted.")
os.chdir(CONTENT_DIR)

Searching for APIs zip file with version 0.1.3 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found matching file: APIs_V0.1.3.zip (ID: 1F6agaMHV1HaEz2pYX5G2GLTXPvjrivQP)

Verifying extracted items:
✅ /content/APIs is present.
✅ /content/DBs is present.
✅ /content/Scripts is present.

✅ Setup complete! Required items extracted to /content.

Generating FC Schemas
✅ Successfully generated 68 FC Schemas to /content/Schemas


## Install Dependencies and Clone Repositories

In [ ]:
!pip install -r /content/APIs/requirements.txt
!pip install freezegun

## Import APIs and initiate DBs

In [ ]:
import google_people
import clock
import notifications
import phone
import device_setting
import re
import freezegun
from freezegun import freeze_time
from datetime import datetime, date, timedelta



def start_frozen_time(current_date):
    """
    Starts a frozen time context using freezegun.
    """
    ignore_pkgs = {"ipykernel", "ipyparallel", "ipython", "jupyter-server"}
    freezegun.configure(extend_ignore_list=list(ignore_pkgs))
    freezer = freeze_time(current_date)
    freezer.start()
    print("--> FROZEN TIME ", datetime.now())
    return freezer


# Variable
current_date = "2025-07-21"
start_frozen_time(current_date)
current_date_dt = datetime.strptime(current_date, "%Y-%m-%d").date()

# Initialize DBs
clock.SimulationEngine.db.load_state("/content/DBs/ClockDefaultDB.json")
google_people.SimulationEngine.db.load_state("/content/DBs/GooglePeopleDefaultDB.json")
notifications.SimulationEngine.db.load_state("/content/DBs/NotificationsDefaultDB.json")
phone.SimulationEngine.db.load_state("/content/DBs/PhoneDefaultDB.json")
device_setting.SimulationEngine.db.load_state("/content/DBs/DeviceSettingDefaultDB.json")

# Add Riya Sharma as a contact
riya_details = {
    "names": [
        {
            "displayName": "Riya Sharma",
            "givenName": "Riya",
            "familyName": "Sharma"
        }
    ],
    "emailAddresses": [
        {
            "value": "riya.sharma@gmail.com"
        }
    ],
    "phoneNumbers": [
        {
            "value": "+1-202-555-0173",
            "type": "mobile"
        },
        {
            "value": "+1-202-555-0199",
            "type": "work"
        }
    ],
    "userDefined": [
        {
            "key": "Nickname",
            "value": "Ri"
        }
    ]
}

google_people.create_contact(riya_details)


# Print the DBs
print(f"Clock DB: {clock.SimulationEngine.db.DB}")
print(f"Google People DB: {google_people.SimulationEngine.db.DB}")
print(f"Notifications DB: {notifications.SimulationEngine.db.DB}")
print(f"Phone DB: {phone.SimulationEngine.db.DB}")

--> FROZEN TIME  2025-07-21 00:00:00
Clock DB: {'alarms': {'ALARM-1': {'alarm_id': 'ALARM-1', 'time_of_day': '7:00 AM', 'date': '2024-01-15', 'label': 'Morning alarm', 'state': 'ACTIVE', 'recurrence': 'MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY', 'created_at': '2024-01-14T22:30:00', 'fire_time': '2024-01-15T07:00:00'}, 'ALARM-2': {'alarm_id': 'ALARM-2', 'time_of_day': '8:30 AM', 'date': '2024-01-15', 'label': 'Meeting reminder', 'state': 'ACTIVE', 'recurrence': '', 'created_at': '2024-01-14T20:15:00', 'fire_time': '2024-01-15T08:30:00'}, 'ALARM-3': {'alarm_id': 'ALARM-3', 'time_of_day': '12:00 PM', 'date': '2024-01-15', 'label': 'Lunch break', 'state': 'ACTIVE', 'recurrence': 'MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY', 'created_at': '2024-01-14T19:45:00', 'fire_time': '2024-01-15T12:00:00'}, 'ALARM-4': {'alarm_id': 'ALARM-4', 'time_of_day': '6:00 PM', 'date': '2024-01-15', 'label': 'Workout time', 'state': 'DISABLED', 'recurrence': '', 'created_at': '2024-01-14T18:20:00', 'fire_time': '2

# Initial Assertion
1. Assert that no 6:00 AM alarm is set for tomorrow.
2. Assert that there are no WhatsApp notifications from Riya.
3. Assert that Riya is a contact.
4. Assert that no call has been placed to Riya yet.
5. Assert that Do Not Disturb is Off.

In [ ]:
import notifications
import google_people
import phone
import clock
import device_setting
from Scripts.assertions_utils import *
from datetime import datetime, date, timedelta
import re

# Local variables
name = "Riya"
dnd_setting = "DO_NOT_DISTURB"
expected_dnd_state = "Off"
current_date = "2025-07-21"
current_date_dt = datetime.strptime(current_date, "%Y-%m-%d").date()
# 1. Assert that no 6:00 AM alarm is set for tomorrow
tomorrow = current_date_dt + timedelta(days=1)
formatted_tmrw = tomorrow.strftime("%Y-%m-%d")
alarms = clock.show_matching_alarms(query="6:00 AM", alarm_type="ACTIVE", date=formatted_tmrw)
alarms_exists = not alarms.get("alarm", [])
assert alarms_exists, "Alarm at 6:00 AM already exists for tomorrow."

# 2. Assert that there are no WhatsApp notifications from Riya.
unread_notifications = notifications.get_notifications(app_name="WhatsApp", unread=True)
pattern = re.compile(rf"\b{name}\b", re.IGNORECASE)
# notification_found = any(bool(pattern.search(bundle['sender']['name'].lower())) for bundle in unread_notifications['bundled_message_notifications'] if compare_strings(bundle['localized_app_name'], 'whatsapp'))
notification_found = any(compare_is_string_subset(name, bundle['sender']['name']) for bundle in unread_notifications['bundled_message_notifications'] if compare_strings(bundle['localized_app_name'], 'whatsapp'))
assert not notification_found, f"There are notifications from {name}, expected to have None."

# 3. Assert that Riya is a contact.
contact_list = google_people.list_connections()
# contact_found = any(bool(pattern.search(n.get('displayName', '').lower())) for c in contact_list['connections'] for n in c.get('names', []))
contact_found = any(compare_is_string_subset(name, n.get('displayName', '')) for c in contact_list.get('connections',[]) for n in c.get('names', []))
assert contact_found, f"There are no contacts for name {name}, expected to have it."

# 4. Assert that no call has been placed to Riya yet.
call_history = phone.utils.get_call_history()
# call_found = any(bool(pattern.search(call['recipient_name'].lower())) for call in call_history.values())
call_found = any(compare_is_string_subset(name, call.get('recipient_name','')) for call in call_history.values())
assert not call_found, f"There are calls to {name}, expected to have None."

# 5. Assert that Do Not Disturb is Off.
dnd_off = compare_strings(device_setting.get(setting_type= dnd_setting).get('on_or_off',''), expected_dnd_state)
assert dnd_off, f"Do Not Disturb is not {expected_dnd_state}, expected to have it."

# Action


**Simulated User**: Schedule Alarm for 6 AM tomorrow.

In [ ]:
# proto_ignore
import clock
import phone
import notifications
import google_people

In [ ]:
clock.create_alarm(
    time="6:00",
    date='2025-07-22'
)

{'message': 'Alarm created successfully for 6:00 AM',
 'action_card_content_passthrough': None,
 'card_id': None,
 'alarm': [{'time_of_day': '6:00 AM',
   'alarm_id': 'ALARM-6',
   'label': '',
   'state': 'ACTIVE',
   'date': '2025-07-22',
   'recurrence': '',
   'fire_time': '2025-07-22T06:00:00'}],
 'timer': None}

**Action Agent**: I have successfully Scheduled alarm for 6 AM tomorrow.

**Simulated User**: Turn on Do Not Disturb so I’m not interrupted.

**Action Agent**: For How long would you like to enable Do Not Disturb?

**Simulated User**: Actually, don't enable it. Any new WhatsApp message from Riya?

In [ ]:
# Checking unread notifications
notifications.get_notifications(app_name="WhatsApp", unread=True)

{'action_card_content_passthrough': None,
 'card_id': None,
 'bundled_message_notifications': [],
 'is_permission_denied': False,
 'status_code': 'OK',
 'skip_reply_disclaimer': None,
 'total_message_count': 0}

**Action Agent**: No, there are no new messages from her.

**Simulated User**: Okay, Call her


In [ ]:
# Checking contact list for Riya
google_people.list_connections()

{'connections': [{'resourceName': 'people/contact_001',
   'etag': 'etag_contact_001',
   'names': [{'metadata': {'primary': True,
      'source': {'type': 'CONTACT', 'id': 'contact_001'}},
     'displayName': 'Alice Johnson',
     'familyName': 'Johnson',
     'givenName': 'Alice'}],
   'emailAddresses': [{'metadata': {'primary': True,
      'source': {'type': 'CONTACT', 'id': 'contact_001'}},
     'value': 'alice.johnson@example.com',
     'type': 'work'}],
   'phoneNumbers': [{'metadata': {'primary': True,
      'source': {'type': 'CONTACT', 'id': 'contact_001'}},
     'value': '+15552345678',
     'type': 'mobile'}],
   'organizations': [{'metadata': {'primary': True,
      'source': {'type': 'CONTACT', 'id': 'contact_001'}},
     'name': 'Design Studio',
     'title': 'Senior Designer',
     'type': 'work'}],
   'created': '2023-02-10T09:15:00Z',
   'updated': '2024-01-10T16:30:00Z'},
  {'resourceName': 'people/contact_002',
   'etag': 'etag_contact_002',
   'names': [{'metadata':

**Action Agent**: Should I call her mobile or work?

**Simulated User**: Call her mobile.

**Action Agent**: Via Phone or Whatsapp?

**Simulated User**: Phone.

In [ ]:
phone.make_call(recipient_name="Riya Sharma",recipient_phone_number="+12025550173")

{'status': 'success',
 'call_id': 'ef4e3a42-e9a6-4d56-a206-5fe8109456eb',
 'emitted_action_count': 1,
 'templated_tts': 'Calling to Riya Sharma at +12025550173.',
 'action_card_content_passthrough': 'Call completed successfully to +12025550173'}

**Action Agent**: Okay, calling Riya on her mobile phone.

# Final Assertion
1. Assert that the 6:00 AM alarm is set for tomorrow.
2. Assert that call has been placed to Riya.
3. Assert that call has been placed to Riya on her mobile
4. Assert that Do Not Disturb is Off.

In [ ]:
import phone
import clock
import google_people
import device_setting
from datetime import datetime, timedelta
from Scripts.assertions_utils import *

# Variable
dnd_setting = "DO_NOT_DISTURB"
current_date = "2025-07-21"
current_date_dt = datetime.strptime(current_date, "%Y-%m-%d").date()

# Local Variable
name = "Riya"
expected_phone_type = "mobile"
expected_dnd_state = "Off"

# 1. Assert that an alarm at 6:00 AM now exists for tomorrow.
tomorrow = current_date_dt + timedelta(days=1)
formatted_tmrw = tomorrow.strftime("%Y-%m-%d")
alarms = clock.show_matching_alarms(query="6:00 AM", alarm_type="ACTIVE", date=formatted_tmrw)
alarm_exists = bool(alarms.get("alarm", []))
assert alarm_exists, "Alarm at 6:00 AM not found for tomorrow."

# 2. Assert that call has been placed to Riya.
call_history = phone.utils.get_call_history()
call_found = any(
    compare_is_string_subset(name, call.get('recipient_name', ''))
    for call in call_history.values()
)
assert call_found, f"There are no calls to {name}, expected to have it."

# 3. Assert that call has been placed to Riya on her mobile
call_on_mobile = False

google_people.search_people(name).get('results',[])[0].get('phoneNumbers')

for phone_number in google_people.search_people(name).get('results',[])[0].get('phoneNumbers',[]):
  if compare_is_string_subset(expected_phone_type, phone_number.get('type')):
    riya_mobile_num = phone_number.get('value','')

for call in call_history.values():
  if compare_is_string_subset(name, call.get('recipient_name', '')):
    if compare_strings(riya_mobile_num, call.get('phone_number', '')):
      call_on_mobile = True

assert call_on_mobile, f"There are no calls to {name} on {expected_phone_type}, expected to have it."

# 4. Assert that Do Not Disturb is Off.
dnd_off = compare_strings(device_setting.get(setting_type= dnd_setting).get('on_or_off',''), expected_dnd_state)
assert dnd_off, f"Do Not Disturb is not {expected_dnd_state}, expected to have it."